
# Advent of Code 2022

> Raise your quality standards as high as you can live with, avoid wasting your time on routine problems, and always try to work as closely as possible at the boundary of your abilities. Do this, because it is the only way of discovering how that boundary should be moved forward.

-- Edsger W. Dijkstra

## Imports and definitions

In [102]:
#type: ignore
from functools import reduce
from operator import mul, and_, add
from dataclasses import dataclass
from collections import defaultdict, Counter, deque
from itertools import product
from enum import Enum
from typing import List


def prod(u): 
    """Older Pythons don't have this."""
    return reduce(mul, u)


def inputfunc(day, kind='lines', testing=False):
    """Generator to read input files."""
    filename = 'test.txt' if testing else f"input/{day}.txt"

    def gen(func):
        if kind == 'lines':
            text = [x.strip() for x in open(filename)]
        elif kind == 'chunks':
            text = [
                x.strip()
                for x in open(filename).read().split('\n\n')
                if x.strip()
            ]
        elif kind == 'single':
            text = open(filename).read().strip()
        elif kind == 'commas':
            text = [x.strip() for x in open(filename).read().split(',')]
        elif kind == 'raw':
            text = open(filename).read()

        def inner():
            return func(f=text)
        return inner
    return gen

## [Day 1 - Calorie Counting](https://adventofcode.com/2022/day/1)

In [2]:
@inputfunc(1, kind='chunks')
def input_1(f):
    return ([int(j) for j in l.split('\n')] for l in f)


A = max(sum(l) for l in input_1())
assert A == 68467


A = sum(sorted((sum(l) for l in input_1()), reverse=True)[:3])
assert A == 203420

## [Day 2 - Rock Paper Scissors](https://adventofcode.com/2022/day/2)

In [3]:
@inputfunc(2)
def input_2(f):
    return (l.split() for l in f)


class Play(Enum):
    ROCK = 1
    PAPER = 2
    SCISSORS = 3

    def __gt__(self, other):
        return (self.value - other.value) % 3 == 1


def score_all(l):
    def score_play(other, me):
        other_play = {
            'A': Play.ROCK,
            'B': Play.PAPER,
            'C': Play.SCISSORS
        }[other]
        my_play = {
            'X': Play.ROCK,
            'Y': Play.PAPER,
            'Z': Play.SCISSORS
        }[me]
        if my_play > other_play:
            return 6 + my_play.value
        elif my_play == other_play:
            return 3 + my_play.value
        else:
            return my_play.value

    return sum(score_play(other, me) for other, me in l)


def score_strat(l):
    def score_play(other, me):
        other_play = {
            'A': Play.ROCK,
            'B': Play.PAPER,
            'C': Play.SCISSORS
        }[other]
        adj = {
            'X': -1,
            'Y': 0,
            'Z': 1
        }[me]
        my_play = next(
            x for x in Play if (x.value % 3) == (other_play.value + adj) % 3
        )
        return my_play.value + 3 * (adj + 1)
    
    return sum(score_play(other, me) for other, me in l)


A = score_all(input_2())
assert A == 15632


A = score_strat(input_2())
assert A == 14416

## [Day 3 - Rucksack Reorganization](https://adventofcode.com/2022/day/3)

In [4]:
@inputfunc(3)
def input_3(f):
    return f


def prio(c):
    if c.isupper():
        return 27 + ord(c) - ord('A')
    else:
        return 1 + ord(c) - ord('a')


def common_split(l):
    def find_common(s):
        mid = len(s) // 2
        return (set(s[:mid]) & set(s[mid:])).pop()

    return sum(prio(find_common(s)) for s in l)


def common_triples(l):
    def find_common(ss):
        return (reduce(and_, ss)).pop()

    def split(ll, k):
        t = [None for _ in range(k)]
        for n, i in enumerate(ll):
            t[n % k] = i
            if n % k == k - 1:
                yield t

    return sum(prio(find_common(set(s) for s in ss)) for ss in split(l, 3))


A = common_split(input_3())
assert A == 7581


A = common_triples(input_3())
assert A == 2525

## [Day 4 - Camp Cleanup](https://adventofcode.com/2022/day/4)

In [5]:
@inputfunc(4)
def input_4(f):
    def parse(l):
        a, b = l.split(',')
        al, ah = a.split('-')
        bl, bh = b.split('-')
        return (int(al), int(ah)), (int(bl), int(bh))

    return (parse(l) for l in f)


def fully_contains(a, b):
    al, ah = a
    bl, bh = b
    return (al <= bl <= bh <= ah) or (bl <= al <= ah <= bh)


def disjoint(a, b):
    al, ah = a
    bl, bh = b
    return (al <= ah < bl <= bh) or (bl <= bh < al <= ah)


A = sum(1 for a, b in input_4() if fully_contains(a, b))
assert A == 511


A = sum(1 for a, b in input_4() if not disjoint(a, b))
assert A == 821

## [Day 5 - Supply Stacks](https://adventofcode.com/2022/day/5)

In [6]:
@dataclass
class Move:
    qty: int
    ori: int
    dest: int


class State:
    def __init__(self, stacks):
        self.stacks = stacks

    def execute(self, i: Move, mode: str = 'lifo'):
        if mode == 'lifo':
            self.stacks[i.dest] += self.stacks[i.ori][-i.qty:][::-1]
        if mode == 'fifo':
            self.stacks[i.dest] += self.stacks[i.ori][-i.qty:]
        self.stacks[i.ori] = self.stacks[i.ori][:-i.qty]


@inputfunc(5, kind='raw')
def input_5(f):
    state, instr = f.split('\n\n')

    # Decode initial state
    boxes = []
    for l in state.split('\n'):
        if not '[' in l:
            break
        boxes.append([l[i] for i in range(1, len(l), 4)])

    stacks = defaultdict(list)
    for l in boxes:
        for n, x in enumerate(l, 1):
            if x != ' ':
                stacks[n].append(x)

    S = State({k: list(reversed(v)) for k, v in stacks.items()})

    # Decode instruction list
    M = []
    for l in instr.split('\n'):
        t = (l.strip()
            .replace('move', '')
            .replace('from', '')
            .replace('to', '')
            .split())
        if len(t) == 3:
            M.append(Move(*(int(c) for c in t)))

    return S, M


def get_final_state(state: State, moves: List[Move], mode='lifo'):
    for m in moves:
        state.execute(m, mode)
    return ''.join(state.stacks[k][-1] for k in sorted(state.stacks))


state, moves = input_5()
A = get_final_state(state, moves, mode='lifo')
assert A == 'CNSZFDVLJ'

state, moves = input_5()
A = get_final_state(state, moves, mode='fifo')
assert A == 'QNDWLMGNS'

## [Day 6 - Tuning Trouble](https://adventofcode.com/2022/day/6)

In [7]:
@inputfunc(6, kind='single')
def input_6(f):
    return f


def first_diff(s, n):
    cnt = Counter()

    for i, c in enumerate(s, 1):
        cnt[c] += 1
        if i > n:
            top = s[i - n - 1]
            cnt[top] -= 1
            if cnt[top] == 0:
                del cnt[top]
        if len(cnt) == n:
            return i


A = first_diff(input_6(), 4)
assert A == 1920


A = first_diff(input_6(), 14)
assert A == 2334

## [Day 7 - No space left on device](https://adventofcode.com/2022/day/7)

In [8]:
@dataclass
class File:
    name: str
    size: int


@dataclass
class Directory:
    name: str
    subdirs: 'List[Directory]'
    files: List[File]
    parent: 'Directory | None'

    def recursive_size(self):
        return sum(f.size for f in self.files) \
            + sum(d.recursive_size() for d in self.subdirs)


@inputfunc(7)
def input_7(f):
    commands = []
    contents = []

    for l in f:
        t = l.split()
        if t[0:2] == ['$', 'cd']:
            commands.extend(contents)
            contents = []
            commands.append(('move', t[2]))
        elif t[0:2] == ['$', 'ls']:
            pass
        elif t[0] == 'dir':
            contents.append(('putdir', t[1]))
        else:
            try:
                contents.append(('putfile', (t[1], int(t[0]))))
            except ValueError:
                pass

    commands.extend(contents)
    return commands


def build_structure(commands):
    root = Directory('/', [], [], None)
    cd = None

    for what, arg in commands:
        if (what, arg) == ('move', '/'):
            cd = root
        elif (what, arg) == ('move', '..'):
           cd = cd.parent
        elif what == 'move':
            l = [d for d in cd.subdirs if d.name == arg]
            if l:
                cd = l[0]
            else:
                nd = Directory(arg, [], [], cd)
                cd.subdirs.append(nd)
        elif what == 'putdir':
            cd.subdirs.append(Directory(arg, [], [], cd))
        elif what == 'putfile':
            cd.files.append(File(*arg))

    return root


def sum_directory_size(root: Directory, maxsize: int):
    own = root.recursive_size()
    capped = own if own <= maxsize else 0
    return capped + sum(sum_directory_size(d, maxsize) for d in root.subdirs)


def minimum_size_above(root: Directory, minsize: int):
    m = (minimum_size_above(d, minsize) for d in root.subdirs)
    return min(
        (x for x in (*m, root.recursive_size()) if x >= minsize),
        default=10**100
    )


R = build_structure(input_7())

A = sum_directory_size(R, 100_000)
assert A == 1391690

A = minimum_size_above(R, R.recursive_size() - 40_000_000)
assert A == 5469168

## [Day 8 - Treetop Tree House](https://adventofcode.com/2022/day/8)

In [9]:
@inputfunc(8)
def input_8(f):
    return [[int(u) for u in x] for x in f]


def visible_trees(M):
    x, y = len(M), len(M[0])
    visible_set = set()

    def max_sofar(it):
        curr_max = None
        for e in it:
            if curr_max is None or e > curr_max:
                curr_max = e
                yield True
            else:
                yield False

    def true_indices(it, ind):
        return (k for e, k in zip(it, ind) if e)

    for i in range(x):
        visible_set |= {(i, j) for j in true_indices(
            max_sofar(M[i][j] for j in range(y)),
            range(y)
        )}

        visible_set |= {(i, j) for j in true_indices(
            max_sofar(M[i][j] for j in reversed(range(y))),
            reversed(range(y))
        )}

    for j in range(y):
        visible_set |= {(i, j) for i in true_indices(
            max_sofar(M[i][j] for i in range(x)),
            range(x)
        )}

        visible_set |= {(i, j) for i in true_indices(
            max_sofar(M[i][j] for i in reversed(range(x))),
            reversed(range(x))
        )}

    return len(visible_set)


def max_scenic_score(M):
    x, y = len(M), len(M[0])
    scores = defaultdict(list)

    def scenic_linear(it):
        st = []
        for n, e in enumerate(it):
            for m, u in reversed(st):
                if u >= e:
                    yield n - m
                    break
            else:
                yield n
            while st and st[-1][1] <= e:
                st.pop()
            st.append((n, e))

    for i in range(x):
        for j, k in zip(
            range(y),
            scenic_linear(M[i][j] for j in range(y))
        ):
            scores[(i, j)].append(k)
        for j, k in zip(
            reversed(range(y)),
            scenic_linear(M[i][j] for j in reversed(range(y)))
        ):
            scores[(i, j)].append(k)

    for j in range(y):
        for i, k in zip(
            range(x),
            scenic_linear(M[i][j] for i in range(x))
        ):
            scores[(i, j)].append(k)
        for i, k in zip(
            reversed(range(x)),
            scenic_linear(M[i][j] for i in reversed(range(x)))
        ):
            scores[(i, j)].append(k)

    return max(prod(w) for w in scores.values())


A = visible_trees(input_8())
assert A == 1859


A = max_scenic_score(input_8())
assert A == 332640

## [Day 9 - Rope Bridge](https://adventofcode.com/2022/day/9)

In [25]:
@inputfunc(9)
def input_9(f):
    return ((w, int(l)) for w, l in (x.split() for x in f))


def visited_squares(L):
    visited = {(0, 0)}
    hx, hy, tx, ty = 0, 0, 0, 0
    for w, l in L:
        if w == 'L':
            hx, hy = hx - l, hy
            dx, dy = abs(hx - tx), abs(hy - ty)
            visited |= {(x, hy) for x in range(tx - 1, hx, -1)}
            if dx > 1 or dy > 1:
                tx, ty = hx + 1, hy
        if w == 'R':
            hx, hy = hx + l, hy
            dx, dy = abs(hx - tx), abs(hy - ty)
            visited |= {(x, hy) for x in range(tx + 1, hx)}
            if dx > 1 or dy > 1:
                tx, ty = hx - 1, hy
        if w == 'U':
            hx, hy = hx, hy + l
            dx, dy = abs(hx - tx), abs(hy - ty)
            visited |= {(hx, y) for y in range(ty + 1, hy)}
            if dx > 1 or dy > 1:
                tx, ty = hx, hy - 1
        if w == 'D':
            hx, hy = hx, hy - l
            dx, dy = abs(hx - tx), abs(hy - ty)
            visited |= {(hx, y) for y in range(ty - 1, hy, -1)}
            if dx > 1 or dy > 1:
                tx, ty = hx, hy + 1
            
    return len(visited)


def extended_visited(L):
    visited = {(0, 0)}
    state = {n: (0, 0) for n in range(10)}
    moves = reduce(add, ([w for _ in range(l)] for w, l in L))
    
    def not_close(a, b):
        ax, ay = a
        bx, by = b
        return abs(ax - bx) > 1 or abs(ay - by) > 1
    
    def step_closer(a, b):
        ax, ay = a
        bx, by = b
        return min(
            ((ax + h, ay + k) for h, k in product((0, 1, -1), (0, 1, -1))),
            key = lambda u: abs(u[0] - bx) + abs(u[1] - by)
        )
        
    for move in moves:
        hx, hy = state[0]
        if move == 'L':
            state[0] = hx - 1, hy
        if move == 'R':
            state[0] = hx + 1, hy       
        if move == 'U':
            state[0] = hx, hy + 1
        if move == 'D':
            state[0] = hx, hy - 1
                    
        for i in range(1, 10):
            tx, ty = state[i-1]
            if not_close(state[i-1], state[i]):
                state[i] = step_closer(state[i], state[i-1])
        
        visited.add(state[9])
            
    return len(visited)


A = visited_squares(input_9())
assert A == 6175


A = extended_visited(input_9())
assert A == 2578

## [Day 10 - Cathode-Ray Tube](https://adventofcode.com/2022/day/10)

In [82]:
@inputfunc(10)
def input_10(f):
    def parse_instruction(s):
        instr, *arg = s.split()
        if instr == 'noop':
            return 'noop',
        if instr == 'addx':
            return 'addx', int(arg[0])

    return [parse_instruction(x) for x in f]


class Interpreter:
    timing = {'noop': 1, 'addx': 2}

    def __init__(self, prg):
        self.ticks = 0
        self.prg = prg
        self.left = self.timing[prg[0][0]]
        self.pc = 0
        self.X = 1
        self.term = False

    def _execute(self):
        instr, *arg = self.prg[self.pc]

        if instr == 'noop':
            pass
        elif instr == 'addx':
            self.X += arg[0]

        self.pc += 1
        if self.pc >= len(self.prg):
            self.term = True
        else:
            self.left = self.timing[self.prg[self.pc][0]]

    def tick(self):
        if self.left == 0:
            self._execute()
        self.left -= 1
        self.ticks += 1


def signal_strength(prg):
    i = Interpreter(prg)
    while not i.term:
        i.tick()
        yield i.X * i.ticks


def crt_draw(prg, columns):
    s = ''
    i = Interpreter(prg)
    while not i.term:
        i.tick()
        beam_col = (i.ticks - 1) % columns
        if abs(i.X - beam_col) <= 1:
            s += '@'
        else:
            s += ' '
        if beam_col == columns - 1:
            s += '\n'
    return s


A = sum(e for n, e in enumerate(signal_strength(input_10()), 1) if n % 40 == 20)
assert A == 13920


print(crt_draw(input_10(), 40))

@@@@  @@  @    @  @ @@@  @    @@@@   @@ 
@    @  @ @    @  @ @  @ @    @       @ 
@@@  @    @    @@@@ @@@  @    @@@     @ 
@    @ @@ @    @  @ @  @ @    @       @ 
@    @  @ @    @  @ @  @ @    @    @  @ 
@@@@  @@@ @@@@ @  @ @@@  @@@@ @     @@  
 


## [Day 11: Monkey in the Middle](https://adventofcode.com/2022/day/11)

In [84]:
@inputfunc(11, testing=True)
def input_11(f):
    ...

## [Day 12 - Hill Climbing Algorithm](https://adventofcode.com/2022/day/12)

In [128]:
@inputfunc(12)
def input_12(f):
    M = [list(x) for x in f]
    start, end = (0, 0), (0, 0)
    for i, row in enumerate(M):
        for j, col in enumerate(row):
            if col == 'S':
                start = (i, j)
                M[i][j] = 'a'
            if col == 'E':
                end = (i, j)
                M[i][j] = 'z'
    return M, start, end


def dfs(M, start, end):
    columns, rows = len(M), len(M[0])
    def neighbors(i, j):
        return {
            (ii, jj) for ii, jj in {(i+1, j), (i-1, j), (i, j+1), (i, j-1)}
            if 0 <= ii < columns and 0 <= jj < rows
        }

    Q = deque((s, 0) for s in start)
    visited = set(start)

    while Q:
        (i, j), steps = Q.popleft()
        for ii, jj in neighbors(i, j):
            if (ii, jj) in visited:
                continue
            if ord(M[ii][jj]) <= 1 + ord(M[i][j]):
                if (ii, jj) == end:
                    return 1 + steps
                Q.append(((ii, jj), 1 + steps))
                visited.add((ii, jj))


M, start, end = input_12()


A = dfs(M, {start}, end)
assert A == 330


A = dfs(M,
    {(i, j) for i, row in enumerate(M) for j, c in enumerate(row) if c == 'a'},
    end
)
assert A == 321